In [326]:
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(8675309)
import copy

In [292]:
def show_examples(x, y):
    plt.figure(figsize=(5,6))
    for i in range(9):
        plt.subplot(3, 3, i+1)
        plt.imshow(x[i], cmap='gray', interpolation='none')
        plt.title("Class {}".format(y[i]))

def norm_shift(count, high=3, stddev=1):
    '''
    Get a shift from a normal distribution. All shifts have an absolute value 1->high. Values are
    chosen from a normal distribution with mean 0 and the input stddev then rounded up to the nearest
    absolute value (ie: 0.5 -> 1 and -0.5 -> -1).
    count -- the number of shifts to return
    high -- clipped maximum, any value from the normal distribution with an absolute value > high
            is clipped to +/- high
    stddev -- the stddev of the normal distribution around 0, does not affect high
    Returns: 1d array of size count with possible values [1, high] both inclusive
    '''
    x = np.random.normal(loc=0, scale=1, size=count).astype(np.float32)
    x_new = (np.clip(np.ceil(np.absolute(x)), 1, high) * np.sign(x)).astype(np.int32)
    return x_new

In [435]:
class Gene(object):
    activations = {
            1: 'relu',
            2: 'selu',
            3: 'tanh',
            4: 'sigmoid'
        }

    def __init__(self, mutate_prob=0.0001):
        self.num_layers = 2
        self.fitness = -1
        # Size of hidden layers, can be 50->1000 by 50
        self.h = np.random.randint(low=1, high=21, size=(self.num_layers), dtype=np.int32)*50
        # Enumerated activation functions, see get_func(a)
        indices = np.random.randint(low=1, high=5, size=(self.num_layers))
        self.a = [activations[i] for i in indices]
        # Dropout rate
        self.d = np.round(np.random.uniform(0.1, 1, size=3), 2)
        # Batch size
        self.b = np.random.randint(low=1, high=7, dtype=np.int32)
        
    def __call__(self):
        from keras.models import Sequential
        from keras.layers.core import Dense, Dropout
        model = Sequential()
        model.add(Dense(self.h[0], activation=self.a[0], input_shape=(784,), name="Dense1"))
        model.add(Dropout(self.d[0], name="Drop1"))
        for i in range(1, self.num_layers):
            model.add(Dense(self.h[i], activation=self.a[i], input_shape=(784,), name="Dense"+str(i+1)))
            model.add(Dropout(self.d[i], name="Drop"+str(i+1)))
        model.add(Dense(10, activation='softmax', name="output"))
        return model
    
    def __str__(self):
        output = []
        output.append('_________________________________________________________________')
        output.append('input\t[size: {:{align}{width}}'.format(784,align='<', width='7')+
                      'batch size: {:{align}{width}}]'.format(2**self.b, align='<', width='25'))
        for i in range(self.num_layers):
            output.append('layer {}\t[size: {:{align}{width}}'.format(i+1, self.h[i],align='<', width='7')+
                          'activation: {:{align}{width}}drop%: {:.2f}]'.format(self.a[i],self.d[i], align='<', width='14'))
        output.append('output\t[size: {:{align}{width}}'.format(10, align='<', width='7')+
                      'activation: {:{align}{width}}]'.format('softmax', align='<', width='25'))
        output.append('_________________________________________________________________')
        return '\n'.join(output)
        
    def mutate(self):
        h_shift = norm_shift(self.num_layers)*np.random.binomial(1, self.mutate_prob, self.num_layers)*50
        self.h = np.clip(self.h+h_shift, 50, 1000)
        a_shift = np.random.binomial(1, self.mutate_prob, self.num_layers)
        for i in range(self.num_layers):
            if a_shift[i]:
                self.a[i] = activations[np.random.randint(low=1, high=5)]
        d_shift = np.random.normal(loc=0, scale=0.1, size=self.num_layers)*np.random.binomial(1, self.mutate_prob, self.num_layers)
        self.d = np.clip(self.d+d_shift, 0.1, 1)
        if np.random.binomial(1, self.mutate_prob):
            self.b = np.clip(self.b+norm_shift(self.num_layers), 1, 6)
            
    def cross(self, other):
        cross_mask = np.random.binomial(1, 0.5, (4, 3))
        new_gene = copy.copy(self)
        for i in range(self.num_layers):
            if cross_mask[0,i]: new_gene.a[i] = other.a[i]
            if cross_mask[1,i]: new_gene.d[i] = other.d[i]
            if cross_mask[2,i]: new_gene.h[i] = other.h[i]
        if cross_mask[3,0]: new_gene.b = other.b
        return new_gene
                
    def batch_size(self):
        return 2**self.b

In [442]:
class Population(object):

    def __init__(self, count):
        from keras.datasets import mnist
        from keras.utils import np_utils
        (x_train, y_train), (x_test, y_test) = mnist.load_data()
        print("X train/test shapes: {} / {}".format(x_train.shape, x_test.shape))
        print("Y train/test shapes: {} / {}".format(y_train.shape, y_test.shape))
        self.x_train = x_train.reshape(x_train.shape[0], 784).astype('float32') / 255
        self.x_test = x_test.reshape(x_test.shape[0], 784).astype('float32') / 255
        self.y_train = np_utils.to_categorical(y_train, 10)
        self.y_test = np_utils.to_categorical(y_test, 10)
        
        self.population = []
        for i in range(count):
            self.population.append(Gene())
    
    def get_fitness(self, i, epochs=10, logging=False):
        from keras import metrics
        from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard
        model = self.population[i]()
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[metrics.categorical_accuracy])
        callback_list = [
            ModelCheckpoint("best_accuracy.hdf5",
                        monitor="val_categorical_accuracy",
                        save_best_only=True, mode='max'),
            CSVLogger("training.log"),
            TensorBoard()
        ] if logging else []
        print("Fitting Gene Model #{}".format(i+1))
        history_callback = model.fit(self.x_train, self.y_train,
                          batch_size=self.population[i].batch_size(), epochs=epochs,
                          verbose=1, validation_split=0.1,
                          callbacks=callback_list)
        if logging: 
            loss_history = history_callback.history["loss"]
            np.savetxt("loss_history.txt", np.array(loss_history), delimiter=",")
        score = model.evaluate(self.x_test, self.y_test, verbose=0)
        #TODO: Add punishment for time to train/model complexity
        self.population[i].fitness = score[1] 
        
    


In [447]:
pop_size = 3
pop = Population(3)
for i in range(pop_size):
    print(pop.population[i])
    pop.get_fitness(i, epochs=2)
for i in range(pop_size):
    print(pop.population[i].fitness)

_________________________________________________________________
input	[size: 784    batch size: 32                       ]
layer 1	[size: 800    activation: relu          drop%: 0.61]
layer 2	[size: 450    activation: tanh          drop%: 0.75]
output	[size: 10     activation: softmax                  ]
_________________________________________________________________
Fitting Gene Model #1
Train on 54000 samples, validate on 6000 samples
Epoch 1/2
54000/54000 [==============================] - 4s 66us/step - loss: 0.3983 - categorical_accuracy: 0.8763 - val_loss: 0.1186 - val_categorical_accuracy: 0.9630
Epoch 2/2
54000/54000 [==============================] - 3s 61us/step - loss: 0.2355 - categorical_accuracy: 0.9305 - val_loss: 0.0963 - val_categorical_accuracy: 0.9698
0.9667
0.9667
_________________________________________________________________
input	[size: 784    batch size: 8                        ]
layer 1	[size: 250    activation: relu          drop%: 0.40]
layer 2	[size: 1